In [33]:
#installed all the required libraries

 !pip install numpy
!pip install panda
!pip install matplotlib
!pip install thefuzz
!pip install seaborn

In [34]:
# Imported the librraries

import re
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from thefuzz import process  # Updated fuzzywuzzy to thefuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Suppress warnings
warnings.filterwarnings('ignore')


In [5]:
#imported the file via google drve

!gdown 12fQqqQ2rt3W_kXX1rHsPBcRR5pPOpR5z

Downloading...
From: https://drive.google.com/uc?id=12fQqqQ2rt3W_kXX1rHsPBcRR5pPOpR5z
To: /content/DA_Task_1.xlsx
100% 33.6k/33.6k [00:00<00:00, 59.9MB/s]


In [7]:
#loaded both the sheets as two different datasets

task_df = pd.read_excel('DA_Task_1.xlsx',sheet_name="Task")
taxonomy_df = pd.read_excel('DA_Task_1.xlsx', sheet_name="Taxonomy")
task_df
taxonomy_df

,Root Cause,Symptom Condition,Symptom Component,Fix Condition,Fix Component
0,Not Tightened,Loose,Cab P Clip,Retightened,Cab P Clip
1,Not Installed,Won't stay open,Fuel Door,Installed,Gas Strut
2,Not Mentioned,Crushed,Compressor Pressure Line,Replaced,Braided Steel
3,Loosened,Oil Running,Not Mentioned,Topped Off,O-Ring
4,Not Included,Missing,Vector,Not Mentioned,Vector
5,Out of Fitting,Oil Dripping,Coupler,Cleaned Out,Coupler
6,Blown,Oil Leak,Mount SVM Sign,Reseted,Brackets
7,Poor Material,Broke,Harness,Repaired,Hydraulic
8,Leaking,Leak,Rinse Tank,Tightened,Not Mentioned
9,Failed Sending,Open,Fuel Sender,NaN,NCV Harness


In [15]:
print(taxonomy_df.columns)



Index(['Root Cause', 'Symptom Condition ', 'Symptom Component',
       'Fix Condition', 'Fix Component'],
      dtype='object')


In [17]:
# removed all the spaces from the column header

taxonomy_df.columns = taxonomy_df.columns.str.strip()
print(taxonomy_df.columns)

Index(['Root Cause', 'Symptom Condition', 'Symptom Component', 'Fix Condition',
       'Fix Component'],
      dtype='object')


In [18]:
# Preprocessing function

def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = text.lower().strip()
    text = re.sub(r'[^a-z0-9 ]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text)  # Normalize spaces
    return text

In [19]:
# Function to find best match

def find_best_match(text, categories):
    if not categories or text == "":
        return "Needs Review"
    match, score = process.extractOne(text, categories)
    return match if score > 80 else "Needs Review"


In [20]:
# Create taxonomy dictionary

taxonomy_dict = {
    "Root Cause": taxonomy_df["Root Cause"].dropna().tolist(),
    "Symptom Condition": taxonomy_df["Symptom Condition"].dropna().tolist(),  # Fix column name
    "Symptom Component": taxonomy_df["Symptom Component"].dropna().tolist(),
    "Fix Condition": taxonomy_df["Fix Condition"].dropna().tolist(),
    "Fix Component": taxonomy_df["Fix Component"].dropna().tolist()
}


In [21]:
# Preprocessing function

def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = text.lower().strip()
    text = re.sub(r'[^a-z0-9 ]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text)  # Normalize spaces
    return text


In [36]:
# Function to find best match


def find_best_match(text, categories):
    if not categories or text == "":
        return "Needs Review"
    match, score = process.extractOne(text, categories)
    return match if score > 80 else "Needs Review"


In [37]:
# Create taxonomy dictionary with corrected column names

taxonomy_dict = {
    "Root Cause": taxonomy_df["Root Cause"].dropna().tolist(),
    "Symptom_Condition": taxonomy_df["Symptom Condition"].dropna().tolist(),  # Trimmed column name
    "Symptom_Component": taxonomy_df["Symptom Component"].dropna().tolist(),
    "Fix_Condition": taxonomy_df["Fix Condition"].dropna().tolist(),
    "Fix_Component": taxonomy_df["Fix Component"].dropna().tolist()
}

In [38]:
# Apply logical tagging based on examples

low_confidence_matches = []
for index, row in task_df.iterrows():
    for tag, categories in taxonomy_dict.items():
        if tag == "Root Cause":
            text = row["Cause"]
        elif "Symptom" in tag:
            text = row["Complaint"]
        elif "Fix" in tag:
            text = row["Correction"]
        else:
            continue

        text = preprocess_text(text)
        best_match = find_best_match(text, categories)
        task_df.at[index, f"{tag}_Tag"] = best_match

        if best_match == "Needs Review":
            low_confidence_matches.append({
                "Index": index,
                "Column": tag,
                "Text": text
            })


In [26]:
# Save tagged data

task_df.to_excel("/content/Tagged_Task_Data.xlsx", index=False)


In [27]:
# Save low-confidence matches for manual review

review_df = pd.DataFrame(low_confidence_matches)
review_df.to_excel("/content/Low_Confidence_Matches.xlsx", index=False)

In [28]:
print("Tagging complete. Files saved:")

Tagging complete. Files saved:


In [29]:
print("- /content/Tagged_Task_Data.xlsx")

- /content/Tagged_Task_Data.xlsx


In [30]:
print("- /content/Low_Confidence_Matches.xlsx")


- /content/Low_Confidence_Matches.xlsx


In [32]:
from google.colab import files

# Download the tagged dataset
files.download("/content/Tagged_Task_Data.xlsx")

# Download the low-confidence matches for review (optional)
files.download("/content/Low_Confidence_Matches.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>